# Class initialisation anti-patterns

Be careful when specifying default args for classes, if you use mutable types and assign default values the same instance of that default value will be assigned to each instance of the class.

In [15]:
# example class that takes a default arg for a mutable type...
class ThisIsBad:
    
    def __init__(self, seems_innocent=[]):
        self.seems_innocent = seems_innocent
        
# create instances...
bad1, bad2 = ThisIsBad(), ThisIsBad()

# both classes start with empty lists...
print(bad1.seems_innocent, bad2.seems_innocent)

# append an element to the list on one of the instances...
bad1.seems_innocent.append("This seems innocent")

# And it appears on both of them!
print(bad1.seems_innocent, bad2.seems_innocent)

[] []
['This seems innocent'] ['This seems innocent']


In [17]:
# we could even create another instance now...
bad3 = ThisIsBad()

# and this will have the mutated list not the expected empty list...
print(bad3.seems_innocent)

# and if we modify this it will change all other instances of this class
# (even those yet to be created)
bad3.seems_innocent.append("But it isn't!")

# here they all are...
print(bad1.seems_innocent, bad2.seems_innocent, bad3.seems_innocent)

['This seems innocent']
['This seems innocent', "But it isn't!"] ['This seems innocent', "But it isn't!"] ['This seems innocent', "But it isn't!"]


## but why ?!?!?!

This is because default args are evaluated once when the function is defined in python, so as soon as the python code that defines a function with a default is run the value is assigned (not once each time the function is called as you may expect).

## Option 1 - Use data classes

One solution is to use data classes with default factories...

In [19]:
# example using data class with a default factory to get a mutable default value without sharing between instances...
from dataclasses import dataclass, field
from typing import List

@dataclass
class ThisIsBetter:
    seems_innocent: List = field(default_factory=lambda: [])
    
# create instances...
better1, better2 = ThisIsBetter(), ThisIsBetter()

# append an element to the field on one instance...
better1.seems_innocent.append('Nice!')

# and it just updates the state on the class that we intended...
print(better1.seems_innocent, better2.seems_innocent)

['Nice!'] []


## Option2 - Use None and assign later

If you want to stick with regular classes or do not have access to python 3.7 features, you can get arround this by assigning the arg to None e.g.

In [22]:
class ThisWillAlsoWork:
    
    def __init__(self, seems_innocent=None):
        self.seems_innocent = [] if not seems_innocent else seems_innocent
        
ok1, ok2 = ThisWillAlsoWork(), ThisWillAlsoWork()

ok1.seems_innocent.append('This will stay in just this instance as intended')

print(ok1.seems_innocent, ok2.seems_innocent)

['This will stay in just this instance as intended'] []
